# Full model code

This notebook contains the **final** model code and docstrings generated by Perplexity.AI between January and March 2024.

The code is stroed in a module called `stroke_rehab.py`. It has been autoformatted using the tool `black`. This was used to aid PEP8 compliance of the code and help with line wrapping.

In [1]:
from IPython.display import Code

## Code stored in `stroke_rehab_model.py`

In [2]:
Code(filename='stroke_rehab_model.py', language='python')

"""A SimPy-based simulation model for analyzing patient flow in stroke care units.

This module simulates the patient flow through an Acute Stroke Unit (ASU) and a connected
Rehabilitation Unit to assist in capacity planning for acute and community stroke services.
It models patient arrivals, treatment durations, and transitions between units using
statistical distributions to provide insights into occupancy levels and potential delays.

Authors: LLM implemented by Perplexity.AI
Auto-formatted with `black` for PEP8 compliance.

Based on:
---------
Monks et al. A modelling tool for capacity planning in acute 
and community stroke services
https://link.springer.com/article/10.1186/s12913-016-1789-4

All functions and classes included in this module have been 
generated between Jan-Mar 2024 by the LLM implemented by Perplexity.AI 
https://www.perplexity.ai/

Auto-formatting:
----------------
After generation was complete We used `black` to autoformat the code
to PEP8 standards

Docstrings:
-----------
After auto-formatting the file was uploaded to Perplexity.AI and the 
following query was issued: 

'Write PEP257 compliant docstrings for all functions, classes and methods.  
Provide a brief description of the purpose of the code, document parameters, 
and return values'

All returned docstrings have been incorporated. This includes the initial
part of the module level docstring above.
"""

import simpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# External function to convert Lognormal moments to Normal moments
def normal_moments_from_lognormal(mean, std_dev):
    """Convert lognormal distribution moments to normal distribution moments.

    Parameters:
    - mean (float): The mean of the lognormal distribution.
    - std_dev (float): The standard deviation of the lognormal distribution.

    Returns:
    - tuple: A tuple containing the mean (mu) and standard deviation (sigma) of the
             corresponding normal distribution.
    """

    phi = np.sqrt(std_dev**2 + mean**2)
    mu = np.log(mean**2 / np.sqrt(std_dev**2 + mean**2))
    sigma = np.sqrt(np.log(phi**2 / mean**2))
    return mu, sigma


class Experiment:
    """A class to set up and run the simulation experiment for stroke patient flow.

    This class initializes the simulation parameters, including patient arrival rates,
    treatment durations, and post-treatment destinations. It also sets up the simulation
    environment and runs the simulation to collect occupancy data for analysis.

    Attributes:
    - Various parameters for initializing the simulation (see __init__ method).

    Methods:
    - reset_kpi: Clears the occupancy data lists.
    - setup_streams: Initializes random number streams for the simulation.
    """

    def __init__(
        self,
        stroke_mean=1.2,
        tia_mean=9.3,
        neuro_mean=3.6,
        other_mean=3.2,
        rehab_mean=7.4,
        rehab_std_dev=8.6,
        esd_mean=4.6,
        esd_std_dev=4.8,
        other_dest_mean=7.0,
        other_dest_std_dev=8.7,
        tia_dest_mean=1.8,
        tia_dest_std_dev=5.0,
        neuro_dest_mean=4.0,
        neuro_dest_std_dev=5.0,
        other_dest_mean_2=3.8,
        other_dest_std_dev_2=5.2,
        results_collection_period=1825,
        trace=False,
        rehab_stroke_iat=21.8,
        rehab_neuro_iat=31.7,
        rehab_other_iat=28.6,
        rehab_stroke_esd_mean=30.3,
        rehab_stroke_esd_std_dev=23.1,
        rehab_stroke_other_mean=28.4,
        rehab_stroke_other_std_dev=27.2,
        rehab_neuro_mean=27.6,
        rehab_neuro_std_dev=28.4,
        rehab_other_mean=16.1,
        rehab_other_std_dev=14.1,
        rehab_stroke_post_destination_prob=[0.4, 0.6],
        rehab_neuro_post_destination_prob=[0.09, 0.91],
        rehab_other_post_destination_prob=[0.12, 0.88],
        warm_up=1095,
        random_number_set=0,
    ):
        self.stroke_interarrival_mean = stroke_mean
        self.tia_interarrival_mean = tia_mean
  